In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('/kaggle/input/digikala-comments-and-products/digikala-comments.csv')
df.head()

In [ ]:
df.info

In [ ]:
df.describe()

In [ ]:
df = df.drop(['id','title','created_at','recommendation_status','is_buyer','product_id','advantages','disadvantages','likes','dislikes','seller_title','seller_code','true_to_size_rate'], axis=1)
df.head()

In [ ]:
df = df.dropna()

In [ ]:
df['rate'] = (df['rate']*20).astype('int')
df.head()

In [ ]:
dfFilter = df.loc[(df['rate']<50) | (df['rate']>=50)]
dfFilter.head()

In [ ]:
dfFilter.loc[:,'label'] = dfFilter['rate'].apply(lambda score: 1 if score>=50 else 0)
dfFilter.head()

In [ ]:
dfFilter.info

In [ ]:
plt.figure(figsize=(8, 6))

plt.bar(['Positive','Negative'], dfFilter['label'].value_counts()/df.shape[0], color=['green', 'red'] )

plt.show()

In [ ]:
count = dfFilter['rate'].value_counts()
fig, axs = plt.subplots(1, 2, figsize=(12, 6), facecolor='white')
palette = sns.color_palette("viridis")
sns.set_palette(palette)
axs[0].pie(count, labels=count.index, autopct='%1.1f%%', startangle=140)
axs[0].set_title('Distribution of Rate')
sns.barplot(x=count.index, y=count.values, ax=axs[1], palette="viridis")
axs[1].set_title('Count of Rate')
plt.tight_layout()
plt.show()

In [ ]:
 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dfFilter['body'].values, dfFilter['label'].values, stratify=dfFilter['label'])
X_train.shape, y_train.shape, X_test.shape, y_test.shape


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

text_processing_pipeline = Pipeline([
    ('Vectorize', CountVectorizer()),
    ('TF-IDF', TfidfTransformer())
])

In [ ]:
text_processing_pipeline

In [ ]:
X_train = text_processing_pipeline.fit_transform(X_train)

X_test = text_processing_pipeline.transform(X_test)

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report , confusion_matrix
import seaborn as sns

In [ ]:
classifiers = {
    "LogisticRegression": LogisticRegression(random_state=42),
#     "SVC": SVC(kernel='linear'),
#     "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),
#     "MultinomialNB": MultinomialNB()
}

for classifier_name, classifier_obj in classifiers.items():
    classifier_obj.fit(X_train, y_train)
    y_pred = classifier_obj.predict(X_test)
    report = classification_report(y_test, y_pred)
    print(f'Classification report for {classifier_name}: ')
    print(report)
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(3,3))
    sns.heatmap(cm, annot=True, fmt=".0f", cmap = 'Blues_r')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

    print("--------------------------------------------")

In [ ]:
!pip install hazm

In [ ]:
from hazm import *
stopwords = stopwords_list()
print(stopwords[-80:])

In [ ]:
def preprocess_text(text):
  normalizer = Normalizer()
  lemmatizer = Lemmatizer()
  normalized_text = normalizer.normalize(text)
  tokens = word_tokenize(normalized_text)
  filtered_text = [word for word in tokens if word not in stopwords]
  lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_text]
  output = " ".join(lemmatized_tokens) 
  return output  

In [ ]:
dfFilter.iloc[2900]

In [ ]:
def testModel(index):
    data  = dfFilter.iloc[index]
    textProsses = preprocess_text(data["body"])
    custom = text_processing_pipeline.transform([textProsses])
    predict = classifiers["LogisticRegression"].predict(custom)
    # "SVC" "RandomForest" "LogisticRegression" "MultinomialNB"
     
    if predict == [1]:  
        print("This comment in Positive ++++ ")
    else:
        print("This comment in Negative ---- ")
        
    print("\n\n" , 'Model predict:' ,predict , "\n\n",'Prossecing text: ', textProsses , "\n\n" ,'Main data: ', "\n\n",  data)

In [ ]:
testModel(2900)

In [ ]:
!pip install ar_wordcloud

In [ ]:
from ar_wordcloud import ArabicWordCloud
import matplotlib.pyplot as plt

def plotWordClod(inputText):
    cleanText = list(inputText.split(" ")) 
    cleanText = list(filter(lambda x: len(x) > 0, cleanText))
    cleanText = cleanText[-8:]
    listToStr = ' '.join([str(elem) for elem in cleanText])
    print(f'THE TOP WORDS FOR TOPIC #{listToStr}')
    awc = ArabicWordCloud(background_color="grey")
    awc.from_text(inputText).to_image()
    wc = awc.from_text(listToStr)
    plt.figure(figsize=(8, 4))
    plt.imshow(awc, interpolation='bilinear')
    plt.axis('off')
    plt.show()

In [ ]:
def predictYourText(inputText):
    textProsses = preprocess_text(inputText)
    textProsses
    custom = text_processing_pipeline.transform([textProsses])
    predict = classifiers["LogisticRegression"].predict(custom) 
    # "SVC" "RandomForest" "LogisticRegression" "MultinomialNB"
     
    if predict == [1]:  
        print("This comment in Positive ++++ ")
    else:
        print("This comment in Negative ---- ")
        
    print("\n\n" , 'Model predict:' ,predict , "\n\n",'Prossecing text: ', textProsses )
    plotWordClod(textProsses)

In [ ]:
predictYourText("من از جنسش خوشم اومد اما خوب هنوز رنگشو دوست ندارم چون پوستم سبزه ست دوسش نداشتم به نظرم باید رنگشو عوض کنن اگر رنگ تیره داشت بهتر بود")

In [ ]:
predictYourText("از دیجیکالا بعید بود این جنس بدرد نخورو بیاره خیلی بوی بدی میداد انگار از پلاستیک آشغال درست شده بود واقعا دیگه خرید نمیکنم در ضمن مامور ارسال هم خیلی بی ادب بود")


In [ ]:
predictYourText("زیاد جالب نبود و مهم اینکه اصلا بدرد من نمیخورد بدترین خرید من بود .حتی گوشه ش شکسته بود انگار جعبه ش رو موش جویده بود خدا لعننتون کنه")

In [ ]:
predictYourText("وای خیلی خوب بود من خیلی دوسش داشتم رنگش آبی بود و من عاشق رنگ آبی آسمونی ام اصلا آبی خیلی خوبه")
